# Spark Dataframes

## Table of Content

<ol style = "type:1">
    <li><a href = "#basics">Spark Dataframe Basics</a></li>
    <li><a href = "#schema">Clarifying Schemas with Spark</a></li>
    <li><a href = "#selectgrab">Selecting Data vs Grabbing Data</a></li>
    <li><a href = "#creatingcol">Creating and Renaming Columns</a></li>
    <li><a href = "#sqlinteract">Using SQL to Interact with the DataFrame</a></li>
    <li><a href = "#ref">References</a></li>
</ol>

## <a name = "basics">Spark Dataframe Basics</a>

In order to work with Spark dataframes, we need to start a Spark session.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

We will read `people.json`.

In [3]:
df = spark.read.json("people.json")

To show the dataframe, we call the .show() method.

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Note that Spark can handle missing data. It automatically replaces missing data with `null`.

We can use the `.printSchema()` method to obtain the data frame's schema.

In [5]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In particular, `age` has data type `long` because of the `null` we saw earlier.

We call `.columns` to get the columns of the dataframe.

In [6]:
df.columns

['age', 'name']

To obtain a statistical summary, we call `.describe()`.

In [7]:
df.describe()

DataFrame[summary: string, age: string, name: string]

It returns a dataframe with

* a column `summary` containing some strings.
* a column `age` containing some strings.
* a column `name` containing some strings.

To see this data frame, we need to call `.show()`.

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## <a name = "schema">Clarifying Schemas with Spark</a>

The schema was inferred correctly for our dataset. In practice, we are expected to clarify the schema---we need to know which columns are strings, integers, etc. This ensures operations are performed correctly on the dataset.

We can handle this with Spark using type tools.

In [9]:
from pyspark.sql.types import (StructField, 
                               StringType, 
                               IntegerType, 
                               StructType
                              )

We create a list `data_schema` of structure fields here. These structure fields take 3 parameter: name, data type, and knowable. For example, consider the structure field

<center>
    <code>
        StructField("age", IntegerType(), True)
    </code>
</center>

This means the column that this relates to is called `"age"`. Then an integer type (i.e., class instance) is passed in. The remaining Boolean value decides if `null` is allowed. In particular, if we have `False` here instead, an error will occur when there is a missing value.

The following is a schema we are expecting for our data set.

In [10]:
data_schema = [StructField("age", 
                           IntegerType(), 
                           True
                          ),
               StructField("name", 
                           StringType(),
                           True
                          )
              ]

We pass `data_schema` into `StructType`, and load `people.json` with our schema.

In [11]:
final_struc = StructType(fields = data_schema)

In [12]:
df = spark.read.json("people.json", 
                     schema = final_struc
                    )

In [13]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



Instead of `long`, `age` now has data type `integer`.

## <a name = "selectgrab">Selecting Data vs Grabbing Data</a>

The thing about Spark is, `df["age"]` is a column object. C.f. the case in `pandas`.

In [14]:
type(df["age"])

pyspark.sql.column.Column

If we want to get a dataframe with this singular column, we need to use the `.select()` method.

In [15]:
df.select("age")

DataFrame[age: int]

In [16]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



The difference here is `df.select("age")` is returning a column, while `df.select("age").show()` is returning a dataframe that contains a single column. The latter is more flexible in practice.

If we want to check the first 2 rows in a dataframe, we can call `df.head()` and then pass in the number of rows.

In [17]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

This is  list of row objects, and we can index this list.

In [18]:
df.head(2)[0]

Row(age=None, name='Michael')

This is the first row in the dataframe.

The reason for having various specialised objects (e.g., column object, row object) is because of Spark's ability to read from a distributed data source and then map that out to distributed computing.

Selecting multiple columns is similar.

In [19]:
df.select(["age", "name"]).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## <a name = "creatingcol">Creating and Renaming Columns</a>

The `.withColumn()` method returns a new dataframe by adding in a column or replacing an existing column.

In [20]:
df.withColumn("newage", df["age"]).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [21]:
df.withColumn("double_age", df["age"] * 2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



Keep in mind that these changes are not permanent on our original dataframe. In order words, `.withColumn()` is not an inplace operation. We would have to save `df.withColumn("double_age", df["age"] * 2).show()` to a new variable.

Renaming columns is similar.

In [22]:
df.withColumnRenamed("age", "my_new_age").show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## <a name = "sqlinteract">Using SQL to Interact with the DataFrame</a>

We register the dataframe as a SQL temporary view

In [23]:
df.createOrReplaceTempView("people")

In [24]:
results = spark.sql("SELECT * FROM people")

In [25]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



A more complicated example.

In [26]:
new_results = spark.sql("SELECT * FROM people WHERE age = 30")

In [27]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## <a name = "ref">References</a>

<ol style = "type:1">
    <li>Jose Portilla. Spark and Python for Big Data with PySpark.</li>
    <li>Apache Spark. <a href = "https://spark.apache.org/docs/latest/api/python/">https://spark.apache.org/docs/latest/api/python/</a>.</li>
</ol>